# Data Preparation

In [1]:
import numpy as np 
import pandas as pd

from datetime import datetime

In [2]:
# Load dataset
dataset_path = 'ass1/data/dataset_mood_smartphone.csv'
df = pd.read_csv(dataset_path, index_col=False).drop(columns='Unnamed: 0')

def clean_dataset(df):
    # from id
    df["id"] = df["id"].str.extract(r"AS14\.(\d{2})")[0].astype(float).astype(int)
    
    # from time (ensure it's in datetime format)
    df["time"] = pd.to_datetime(df["time"], format="%Y-%m-%d %H:%M:%S.%f")  # Keep as datetime

    # Extract required columns
    df["date"] = df["time"].dt.date  # Extract date
    df["day_of_the_week"] = df["time"].dt.day_name()  # Extract day name
    df["hour_of_the_day"] = df["time"].dt.hour  # Extract hour

    # from variable
    df['record_type'] = df['variable'].apply(lambda x: {'appCat': 'app', 'circumplex': 'sensor'}.get(x.split('.')[0], x.split('.')[0]))
    df['record_detail'] = df['variable'].apply(lambda x: x.split('.')[1] if len(x.split('.')) > 1 else None)

    # select relevant columns
    relevant_cols = [
        'id',
        'time',
        'date',
        'day_of_the_week',
        'hour_of_the_day',
        'hour_of_the_day',
        'record_type',
        'record_detail',
        'value']
    
    return df

# Load dataset
dataset_path = 'ass1/data/dataset_mood_smartphone.csv'
df = pd.read_csv(dataset_path, index_col=False).drop(columns='Unnamed: 0')

# Clean dataset
df = clean_dataset(df)

# Save cleaned dataset
cleaned_dataset_path = 'ass1/data/dataset_clean.csv'
df.to_csv(cleaned_dataset_path, index=False)